Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [5]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [6]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [96]:
beta = 0.001
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
     
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
    
  # RELU Layer
  hidden_layer = tf.nn.relu(logits)
  hidden_weights = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  hidden_biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden_layer, hidden_weights) + hidden_biases
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))
     
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_logits = tf.matmul(valid_logits, hidden_weights) + hidden_biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_logits = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_logits = tf.matmul(test_logits, hidden_weights) + hidden_biases
  test_prediction = tf.nn.softmax(test_logits)

In [97]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 660.379150
Minibatch accuracy: 3.1%
Validation accuracy: 26.6%
Minibatch loss at step 500: 295.066467
Minibatch accuracy: 81.2%
Validation accuracy: 78.4%
Minibatch loss at step 1000: 260.059082
Minibatch accuracy: 89.1%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 238.110657
Minibatch accuracy: 82.0%
Validation accuracy: 82.0%
Minibatch loss at step 2000: 213.149338
Minibatch accuracy: 83.6%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 189.864975
Minibatch accuracy: 89.1%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 173.300629
Minibatch accuracy: 82.8%
Validation accuracy: 83.8%
Test accuracy: 90.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [98]:
beta = 0.001
batch_size = 5
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
     
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
    
  # RELU Layer
  hidden_layer = tf.nn.relu(logits)
  hidden_weights = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  hidden_biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden_layer, hidden_weights) + hidden_biases
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))
     
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_logits = tf.matmul(valid_logits, hidden_weights) + hidden_biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_logits = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_logits = tf.matmul(test_logits, hidden_weights) + hidden_biases
  test_prediction = tf.nn.softmax(test_logits)

In [99]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 669.060730
Minibatch accuracy: 0.0%
Validation accuracy: 17.8%
Minibatch loss at step 500: 868.809204
Minibatch accuracy: 40.0%
Validation accuracy: 73.4%
Minibatch loss at step 1000: 495.121887
Minibatch accuracy: 40.0%
Validation accuracy: 68.1%
Minibatch loss at step 1500: 303.379669
Minibatch accuracy: 60.0%
Validation accuracy: 73.0%
Minibatch loss at step 2000: 215.074661
Minibatch accuracy: 100.0%
Validation accuracy: 72.0%
Minibatch loss at step 2500: 218.544937
Minibatch accuracy: 80.0%
Validation accuracy: 73.1%
Minibatch loss at step 3000: 176.467590
Minibatch accuracy: 100.0%
Validation accuracy: 71.9%
Test accuracy: 79.4%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [102]:
beta = 0.001
batch_size = 5
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
     
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
    
  # RELU Layer
  hidden_layer = tf.nn.relu(logits)
  hidden_weights = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  hidden_biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden_layer, hidden_weights) + hidden_biases

  keep_prob = tf.placeholder("float")
  logits = tf.nn.dropout(logits, keep_prob)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))
     
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_logits = tf.matmul(valid_logits, hidden_weights) + hidden_biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_logits = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_logits = tf.matmul(test_logits, hidden_weights) + hidden_biases
  test_prediction = tf.nn.softmax(test_logits)

In [103]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.7}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 598.379517
Minibatch accuracy: 40.0%
Validation accuracy: 14.8%
Minibatch loss at step 500: 1452.923096
Minibatch accuracy: 40.0%
Validation accuracy: 70.8%
Minibatch loss at step 1000: 359.015015
Minibatch accuracy: 80.0%
Validation accuracy: 70.7%
Minibatch loss at step 1500: 265.687622
Minibatch accuracy: 60.0%
Validation accuracy: 70.8%
Minibatch loss at step 2000: 285.593750
Minibatch accuracy: 60.0%
Validation accuracy: 70.0%
Minibatch loss at step 2500: 222.036194
Minibatch accuracy: 40.0%
Validation accuracy: 69.9%
Minibatch loss at step 3000: 196.159927
Minibatch accuracy: 80.0%
Validation accuracy: 75.1%
Test accuracy: 82.6%


Some of the overfitting is mitigated by using dropout

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [21]:
beta = 0.001
batch_size = 128
num_hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
     
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_hidden_nodes]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
    
  # RELU Layer
  hidden_layer = tf.nn.relu(logits)
  hidden_weights = tf.Variable(tf.truncated_normal([num_hidden_nodes, num_labels]))
  hidden_biases = tf.Variable(tf.zeros([num_labels]))
  logits = tf.matmul(hidden_layer, hidden_weights) + hidden_biases

  keep_prob = tf.placeholder("float")
  logits = tf.nn.dropout(logits, keep_prob)
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))
     
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_logits = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)
  valid_logits = tf.matmul(valid_logits, hidden_weights) + hidden_biases
  valid_prediction = tf.nn.softmax(valid_logits)
    
  test_logits = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
  test_logits = tf.matmul(test_logits, hidden_weights) + hidden_biases
  test_prediction = tf.nn.softmax(test_logits)

In [22]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.7}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 779.549194
Minibatch accuracy: 5.5%
Validation accuracy: 23.0%
Minibatch loss at step 500: 306.769958
Minibatch accuracy: 65.6%
Validation accuracy: 80.6%
Minibatch loss at step 1000: 261.906128
Minibatch accuracy: 69.5%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 245.110016
Minibatch accuracy: 68.8%
Validation accuracy: 80.7%
Minibatch loss at step 2000: 218.603027
Minibatch accuracy: 66.4%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 193.488083
Minibatch accuracy: 67.2%
Validation accuracy: 84.1%
Minibatch loss at step 3000: 176.729309
Minibatch accuracy: 53.9%
Validation accuracy: 84.4%
Test accuracy: 90.7%
